In [ ]:
import polars as pl

In [ ]:
siafi=(
    pl.read_csv('./municipio_igbe_siafi.csv')
    .rename({
        'codigo_ibge':'municipio_codigo_7',
    })
    .drop([
        'codigo_uf',
        'fuso_horario'
    ])
    .with_columns(
        pl.col('municipio_codigo_7').cast(str)
    )
)
siafi

In [ ]:
(
    pl.read_excel('./territorio_codigo.xlsx')
    .filter(
        pl.col('categoria') == 'Município'
    )
    .drop(['categoria'])
    .rename({
        'codigo':'municipio_codigo_7',
        'nome':'municipio_nome',
    })
    .with_columns(
        pl.col('municipio_codigo_7').str.slice(length=2, offset=0).alias('uf_codigo')
    )
    .join(
        other=pl.read_parquet('./uf_completo.parquet'),
        on='uf_codigo',
        how='left'
    )
    .join(
        other=siafi,
        on='municipio_codigo_7',
        how='left'
    )
    .drop(['nome'])
    .rename({
        'siafi_id':'municipio_siafi'
    })
).write_parquet('./municipio_completo.parquet')